In [1]:
## final annotations preparation
import pandas as pd

anno1200_df=pd.read_csv(r'C:\Users\gen80\OneDrive\Documents\MDSlectures\COLX_523_advanced_corp_ling\COLX523_SH_VT_AL\milestone3\Batch_3948609_full2074_results.csv')

#anno1200_df=pd.read_csv(r'C:\Users\gen80\OneDrive\Documents\MDSlectures\COLX_523_advanced_corp_ling\COLX523_SH_VT_AL\milestone3\1200annos.csv')

In [2]:
anno1200_df.keys()
#anno1200_df.iloc[1]

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.company',
       'Input.text',
       'Answer.对提到的科技企业表达的情绪 sentiment analysis on tech companies.label',
       'Approve', 'Reject'],
      dtype='object')

In [3]:
anno1200_df =anno1200_df.rename(columns={'Answer.对提到的科技企业表达的情绪 sentiment analysis on tech companies.label':'label'})

In [4]:
anno1200_df = anno1200_df.query("AssignmentStatus != 'Rejected' ")
len(anno1200_df)

1187

In [5]:
anno1200_df['label']=anno1200_df['label'].str.replace("十分正面|正面|中性|负面|十分负面|无关","",regex=True)
anno1200_selected = anno1200_df[['HITId','WorkerId','Input.text','label']]
anno1200_selected.head()

,HITId,WorkerId,Input.text,label
0,301KG0KXAGOVAAB67J6QDSZBS1WH23,A1RP234EJ8NH7K,200219 #Logan Lerman# attends the PC of Amazon...,Irrelevant
2,301KG0KXAGOVAAB67J6QDSZBS1WH23,A2EWT74VHA63LE,200219 #Logan Lerman# attends the PC of Amazon...,Irrelevant
7,302U8RURK34WHXVJYRLEKHTIBBYVNY,A1RP234EJ8NH7K,新冠病毒扩散或让Netflix受益！ 据Variety，BMO资本市场分析师丹·萨蒙指出，新...,Irrelevant
9,302U8RURK34WHXVJYRLEKHTIBBYVNY,ACOLIDKYK5VNO,新冠病毒扩散或让Netflix受益！ 据Variety，BMO资本市场分析师丹·萨蒙指出，新...,Neutral
10,302U8RURK34WHXVJYRLEKHTIBBYVNY,A3D7RB6M40R8FM,新冠病毒扩散或让Netflix受益！ 据Variety，BMO资本市场分析师丹·萨蒙指出，新...,Positive


In [6]:
len(anno1200_selected)

1187

In [7]:
from collections import defaultdict, Counter
label_per_task = defaultdict(list)
for i in range(len(anno1200_selected)):
    label_per_task[anno1200_selected.iloc[i]['HITId']].append(anno1200_selected.iloc[i]['label'])

In [8]:
label_per_task

defaultdict(list,
            {'301KG0KXAGOVAAB67J6QDSZBS1WH23': ['Irrelevant', 'Irrelevant'],
             '302U8RURK34WHXVJYRLEKHTIBBYVNY': ['Irrelevant',
              'Neutral',
              'Positive'],
             '304QEQWK0TNY6HUFW9X2ED3X7PR0OR': ['Negative',
              'Negative',
              'Very Negative'],
             '307L9TDWK2VC5OT0EHF81M1ZKR63NA': ['Irrelevant',
              'Irrelevant',
              'Neutral'],
             '30F94FBDOVNF3W2XQ6GQS453X14TBR': ['Negative',
              'Negative',
              'Negative'],
             '30IRMPJWE3M3GKTZSUOORSNBWAGRKW': ['Irrelevant',
              'Negative',
              'Neutral'],
             '30QQTY5GNON71XEWDUUKSY59XPKU7X': ['Irrelevant',
              'Irrelevant',
              'Neutral'],
             '30U1YOGZHEZ73TN2F226DPXHY32DST': ['Very Negative',
              'Very Negative',
              'Negative'],
             '30Y6N4AHZTZV2ENP0D21NFSLC1JDR5': ['Irrelevant',
              'Irrelevant',
 

In [9]:
###rules for final annotations
            
##when there's three annotators(almost all entries)
def majority_rule(label_list): ##take a majority rule
    majority_agree = 0
    label_counter = Counter(label_list)
    annot_dict = {"Very Positive": 5, "Positive": 4, "Neutral": 3, "Negative": 2, "Very Negative": 1}
    rev_dict = dict([(v,k) for k,v in annot_dict.items()])
    # if 2 or 3 annotators agree on a particular label, pick it
    if max(label_counter.values()) >=2:
        final = max(label_counter, key =label_counter.get)
        majority_agree = 1
    else: # if all 3 annotators label differently, take the average sentiment except for "Irrelevant"
        if "Irrelevant" not in label_list:
            avg_sentiment = sum([annot_dict[i] for i in label_list])/len(label_list)
            final = rev_dict[round(avg_sentiment)]
            #print(final)
        else:
            final = "Irrelevant"
        
    return final,majority_agree    
    
##when there's two annotators
def label_2(an1,an2):
    if an1 == an2:
        final = an1
    elif an1 == "Irrelevant" or an2 == "Irrelevant":
        final = "Irrelevant"
    elif an1 == "Neutral" or an2 == "Neutral":
        final = "Neutral"
    elif an1.startswith("Very") or an2.startswith("Very"):
        if an1[5] == "N" or an2[5] == "N":
            final = "Negative"
        if an1[5] == "P" or an2[5] == "P":
            final = "Positive"
    elif an1 == "Negative" and an2 == "Positive" or an1 == "Positive" and an2 == "Negative":
        final = "Neutral"
    return final
    
count = 0
majority_count = 0
finallabels = dict()
for k,v in label_per_task.items():
    
    count +=1
    print("count:",count)
    print(v)
    if len(v) ==3:
        print(majority_rule(v))
        finallabels[k] = majority_rule(v)[0]
        majority_count += majority_rule(v)[1]
        
    elif len(v) ==2:
        an1, an2 = v
        print(label_2(an1,an2))
        finallabels[k] = label_2(an1,an2)

        

count: 1
['Irrelevant', 'Irrelevant']
Irrelevant
count: 2
['Irrelevant', 'Neutral', 'Positive']
('Irrelevant', 0)
count: 3
['Negative', 'Negative', 'Very Negative']
('Negative', 1)
count: 4
['Irrelevant', 'Irrelevant', 'Neutral']
('Irrelevant', 1)
count: 5
['Negative', 'Negative', 'Negative']
('Negative', 1)
count: 6
['Irrelevant', 'Negative', 'Neutral']
('Irrelevant', 0)
count: 7
['Irrelevant', 'Irrelevant', 'Neutral']
('Irrelevant', 1)
count: 8
['Very Negative', 'Very Negative', 'Negative']
('Very Negative', 1)
count: 9
['Irrelevant', 'Irrelevant', 'Positive']
('Irrelevant', 1)
count: 10
['Negative', 'Very Negative', 'Very Negative']
('Very Negative', 1)
count: 11
['Positive', 'Negative', 'Positive']
('Positive', 1)
count: 12
['Irrelevant', 'Irrelevant', 'Irrelevant']
('Irrelevant', 1)
count: 13
['Irrelevant', 'Neutral', 'Irrelevant']
('Irrelevant', 1)
count: 14
['Negative', 'Negative', 'Neutral']
('Negative', 1)
count: 15
['Neutral', 'Positive', 'Positive']
('Positive', 1)
count: 16

In [10]:
len(finallabels)

407

In [11]:
majority_count

279

In [12]:
anno1200_df.query("HITId=='301KG0KXAGOVAAB67J6QDSZBS1WH23' ")["Input.text"].unique()[0]

"200219 #Logan Lerman# attends the PC of Amazon Prime Video's &quot;Hunters&quot;"

In [13]:
label_df = pd.DataFrame(finallabels.values(), index=finallabels.keys(),columns= ["FinalLabel"])

In [14]:
#merge the final rating to the original dataframe

Id_df = anno1200_df.set_index('HITId')

In [15]:
merge_df =pd.merge(Id_df,label_df, left_index = True, right_index = True)

In [16]:
finallabel_df = merge_df[['Input.company','Input.text','FinalLabel']]

In [17]:
finallabel_df.head()

,Input.company,Input.text,FinalLabel
301KG0KXAGOVAAB67J6QDSZBS1WH23,Amazon,200219 #Logan Lerman# attends the PC of Amazon...,Irrelevant
301KG0KXAGOVAAB67J6QDSZBS1WH23,Amazon,200219 #Logan Lerman# attends the PC of Amazon...,Irrelevant
302U8RURK34WHXVJYRLEKHTIBBYVNY,Facebook,新冠病毒扩散或让Netflix受益！ 据Variety，BMO资本市场分析师丹·萨蒙指出，新...,Irrelevant
302U8RURK34WHXVJYRLEKHTIBBYVNY,Facebook,新冠病毒扩散或让Netflix受益！ 据Variety，BMO资本市场分析师丹·萨蒙指出，新...,Irrelevant
302U8RURK34WHXVJYRLEKHTIBBYVNY,Facebook,新冠病毒扩散或让Netflix受益！ 据Variety，BMO资本市场分析师丹·萨蒙指出，新...,Irrelevant


In [18]:
finallabel_df =finallabel_df.drop_duplicates()

In [19]:
finallabel_df.head()

,Input.company,Input.text,FinalLabel
301KG0KXAGOVAAB67J6QDSZBS1WH23,Amazon,200219 #Logan Lerman# attends the PC of Amazon...,Irrelevant
302U8RURK34WHXVJYRLEKHTIBBYVNY,Facebook,新冠病毒扩散或让Netflix受益！ 据Variety，BMO资本市场分析师丹·萨蒙指出，新...,Irrelevant
304QEQWK0TNY6HUFW9X2ED3X7PR0OR,Amazon,#亚马逊再曝售辱华商品# 还记得去年8月被扒出在平台上售卖多款“gangdu”T恤吗？还没凉...,Negative
307L9TDWK2VC5OT0EHF81M1ZKR63NA,Facebook,#想见你# #想见你彩蛋接力赛# 柯佳嬿 许光汉 施柏宇 颜毓麟 无水印 图源施柏宇脸书 ...,Irrelevant
30F94FBDOVNF3W2XQ6GQS453X14TBR,Amazon,看了那么多则举报口罩虚假销售问题，没想到我也能碰上。 作为亚马逊付费会员，我在1月24日在亚...,Negative


In [20]:
ready_df = finallabel_df.reset_index().drop(columns=["index"]) #this final output is largely correct now

In [21]:
ready_df=ready_df.rename(columns={"Input.company":"Company","Input.text":"Text"})

In [22]:
ready_df.to_csv("Weibo_407annotations_final.csv",index=False,encoding="utf-8")